<a href="https://colab.research.google.com/github/nolpen6/MushroomInsight_ML/blob/main/notebooks/01_mushroom_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>